In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime,timedelta


In [ ]:
url = 'https://groww.in/v1/api/stocks_data/v1/all_stocks'
headers = {
    'Content-Type': 'application/json',
}
symList = []
for i in range(3):
    payload = {
        "listFilters": {
            "INDUSTRY": [],
            "INDEX": []
        },
        "objFilters": {
            "CLOSE_PRICE": {"max": 100000, "min": 0},
            "MARKET_CAP": {"min": 0, "max": 2000000000000000}
        },
        "page": i,
        "size": "2000",
        "sortBy": "NA",
        "sortType": "ASC"
    }
    
    
    
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        symList.extend(response.json()['records'])
        print(len(symList))
    else:
    
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)
        
sectordf = pd.json_normalize(symList)
sectordf

In [ ]:
sectordf['marketCap'] = sectordf['marketCap']/1000000000
sectordf = sectordf[sectordf.marketCap > 500]
sectordf

In [ ]:
secotrClassUrl = 'https://groww.in/v1/api/stocks_data/v1/all_stocks/filtersV2'
response = requests.get(secotrClassUrl)
industryJson= response.json()
industryJson

In [ ]:
table_data = []

# Iterate through the JSON data to extract sector, industry, and industry code
for item in industryJson['filterData']['INDUSTRY']:
    sector = item['sector']
    industries = item['industries']
    for code, industry in industries.items():
        table_data.append({'sector': sector, 'industry': industry, 'industryCode': int(code)})
table_data

In [ ]:
# Create a DataFrame from the extracted data
df = pd.DataFrame(table_data)
df = df.sort_values(by='industryCode')
df.reset_index(drop=True,inplace =True)
finalsectorData = sectordf.merge(df, on = 'industryCode')
finalsectorData = finalsectorData[['isin','companyName','industryCode','bseScriptCode','nseScriptCode','sector','industry','marketCap']]
finalsectorData

In [ ]:
finalsectorData.to_csv("sectorClass.csv")

## Display Through Panel

In [ ]:
pip install panel

In [ ]:
import panel as pn
pn.extension()
df = finalsectorData

# Create a Panel DataFrame
pn_df = pn.widgets.DataFrame(df, name='DataFrame',width=1000)

# Define the dropdown options as a list
dropdown_options = list(df['sector'].unique())

# Define the dropdown widget
dropdown = pn.widgets.Select(name='Filter by Sector', options=dropdown_options)

defaultIndustryList = df[df['sector'] == df['sector'].unique()[0]]['industry'].unique().tolist()
dropdown2 = pn.widgets.Select(name='Filter by Industry', options=defaultIndustryList)

# Define a function to update the displayed DataFrame based on the selected category
def update_dataframe(event):
    selected_category = dropdown.value
    filtered_df = df[df['sector'] == selected_category]
   
    filtered_industries = df[df['sector'] == selected_category]['industry'].unique().tolist()
    dropdown2.options = filtered_industries
    pn_df.value = filtered_df

def update_dataframe2(event):
    selected_category = dropdown2.value
    filtered_df = df[df['industry'] == selected_category]
    pn_df.value = filtered_df

# Set up the interaction between the dropdown and dataframe
dropdown.param.watch(update_dataframe, 'value')

dropdown2.param.watch(update_dataframe2, 'value')

# Create a panel dashboard layout
dashboard = pn.Column(
    '## Dashboard',
    dropdown, dropdown2,
    pn_df
)


dashboard